In [3]:
import keras_tuner as kt
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [4]:
#  Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(["EIN", "NAME"], axis=1, inplace=True)

In [6]:

# Determine the number of unique values in each column.
filtered_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
filtered_cat

['APPLICATION_TYPE',
 'AFFILIATION',
 'CLASSIFICATION',
 'USE_CASE',
 'ORGANIZATION',
 'INCOME_AMT',
 'SPECIAL_CONSIDERATIONS']

In [7]:
application_df[filtered_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [8]:
# Look at APPLICATION_TYPE value counts for binning
app_type = application_df['APPLICATION_TYPE'].value_counts().reset_index()
app_type

,index,APPLICATION_TYPE
0,T3,27037
1,T4,1542
2,T6,1216
3,T5,1173
4,T19,1065
5,T8,737
6,T7,725
7,T10,528
8,T9,156
9,T13,66


In [9]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = []
application_types_to_replace = app_type[app_type['APPLICATION_TYPE'] <= 156]['index']

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [10]:

# Look at CLASSIFICATION value counts for binning
class_count = application_df['CLASSIFICATION'].value_counts().reset_index()
class_count

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
...,...,...
66,C4120,1
67,C8210,1
68,C2561,1
69,C4500,1


In [11]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_filtered = []
class_count_filtered = class_count[class_count['CLASSIFICATION'] > 1]
class_count_filtered

,index,CLASSIFICATION
0,C1000,17326
1,C2000,6074
2,C1200,4837
3,C3000,1918
4,C2100,1883
5,C7000,777
6,C1700,287
7,C4000,194
8,C5000,116
9,C1270,114


In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = []
classifications_to_replace = class_count[class_count['CLASSIFICATION'] < 1883]['index']

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()
     

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1


In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
numerical_df = pd.get_dummies(application_df)
numerical_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
# Split our preprocessed data into our features and target arrays
y = numerical_df["IS_SUCCESSFUL"].values
X = numerical_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

C:\Users\18162\AppData\Local\Temp\ipykernel_7616\120369778.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = numerical_df.drop(["IS_SUCCESSFUL"],1).values


In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [17]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=120,
        step=5), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=120,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [18]:
# Import the kerastuner library

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [19]:

# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 42s]
val_accuracy: 0.7314285635948181

Best val_accuracy So Far: 0.7343440055847168
Total elapsed time: 00h 18m 20s
INFO:tensorflow:Oracle triggered exit


In [20]:

# Get top 3 model hyperparameters and print the values
top_hyper = tuner.get_best_hyperparameters(3)
for param in top_hyper:
    print(param.values)

{'activation': 'relu', 'first_units': 41, 'num_layers': 3, 'units_0': 81, 'units_1': 106, 'units_2': 101, 'units_3': 96, 'units_4': 36, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
{'activation': 'relu', 'first_units': 36, 'num_layers': 3, 'units_0': 41, 'units_1': 16, 'units_2': 16, 'units_3': 6, 'units_4': 26, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'activation': 'tanh', 'first_units': 61, 'num_layers': 5, 'units_0': 76, 'units_1': 6, 'units_2': 86, 'units_3': 26, 'units_4': 56, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 1, 'tuner/round': 1, 'tuner/trial_id': '0049'}


In [21]:
# Evaluate the top 3 models against the test dataset
top_model = tuner.get_best_models(3)
for model in top_model:
    model_loss, model_accuracy = model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5529 - accuracy: 0.7343 - 488ms/epoch - 2ms/step
Loss: 0.5528819561004639, Accuracy: 0.7343440055847168
268/268 - 0s - loss: 0.5556 - accuracy: 0.7341 - 493ms/epoch - 2ms/step
Loss: 0.5555933117866516, Accuracy: 0.7341107726097107
268/268 - 1s - loss: 0.5541 - accuracy: 0.7339 - 501ms/epoch - 2ms/step
Loss: 0.5541064143180847, Accuracy: 0.7338775396347046


In [22]:
# Creating model based on the top hyperparameter

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu', input_dim = 43))
# First hidden layer
nn.add(tf.keras.layers.Dense(units = 56, activation = 'relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 71, activation = 'relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 11, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 96, activation = 'relu'))

nn.add(tf.keras.layers.Dense(units = 31, activation = 'relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs =20)

Epoch 1/20
804/804 [==============================] - 3s 2ms/step - loss: 0.5841 - accuracy: 0.7108
Epoch 2/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5599 - accuracy: 0.7257
Epoch 3/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5548 - accuracy: 0.7273
Epoch 4/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5524 - accuracy: 0.7305
Epoch 5/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7322
Epoch 6/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7313
Epoch 7/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5482 - accuracy: 0.7310
Epoch 8/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5475 - accuracy: 0.7324
Epoch 9/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7330
Epoch 10/20
804/804 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7328

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5572 - accuracy: 0.7272 - 577ms/epoch - 2ms/step
Loss: 0.5571720600128174, Accuracy: 0.7272303104400635


In [26]:

# Exporting the model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')
